#Move first chunk


In [ ]:
%%capture
!apt-get install lzop

In [ ]:
!cp "drive/MyDrive/twitter/onechunk/part-00000.lzo" file.lzo

In [ ]:
!lzop -x file.lzo
!mv file Dataset.csv

In [ ]:
import gc
import numpy as np
np.random.seed(1234)
gc.collect()
all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "engaging_user_follower_count", "engaging_user_following_count", "engaging_user_is_verified",\
               "engaging_user_account_creation", "engagee_follows_engager","reply_timestamp", "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]
all_features_to_idx = dict(zip(all_features, range(len(all_features))))

#Load with pandas and train/test splitting

In [ ]:
import pandas as pd
df= pd.read_csv("Dataset.csv",sep='\x01',header=None)
df.columns=all_features

In [ ]:
gc.collect()

3383989

##Target classes encode

In [ ]:
df["like"]=df["like_timestamp"].apply(lambda row: 1 if row>0 else 0).astype("uint8")
df.drop("like_timestamp",axis=1,inplace=True)

df["retweet"]=df["retweet_timestamp"].apply(lambda row: 1 if row>0 else 0).astype("uint8")
df.drop("retweet_timestamp",axis=1,inplace=True)

df["retweetCom"]=df["retweet_with_comment_timestamp"].apply(lambda row: 1 if row>0 else 0).astype("uint8")
df.drop("retweet_with_comment_timestamp",axis=1,inplace=True)

df["reply"]=df["reply_timestamp"].apply(lambda row: 1 if row>0 else 0).astype("uint8")
df.drop("reply_timestamp",axis=1,inplace=True)
gc.collect()
df["interaction"]=df["reply"]+df["like"]+df["retweet"]+df["retweetCom"]
df["interaction"]=df["interaction"].astype("uint8")

##Divide media types

In [ ]:
def countMedia(arr,mediaType):
    c=0
    for m in arr:
        if m==mediaType:
            c+=1
    return c
df.fillna(value={"present_media":""},inplace=True)
df["photos"]=df["present_media"].apply(lambda row: countMedia((row.split("\t")),"Photo")).astype("uint8")
df["gifs"]=df["present_media"].apply(lambda row: countMedia((row.split("\t")),"GIF")).astype("uint8")
df["videos"]=df["present_media"].apply(lambda row: countMedia((row.split("\t")),"Video")).astype("uint8")
gc.collect()


53

##Replace text tokens , link, media etc with their count

In [ ]:
df["text_tokens"]=df["text_tokens"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint16")
df.fillna(value={"hashtags":""},inplace=True)
df["hashtags"]=df["hashtags"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint8")

df["present_media"]=df["present_media"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint8")
df.fillna(value={"present_links":""},inplace=True)
df["present_links"]=df["present_links"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint8")
df.fillna(value={"present_domains":""},inplace=True)
df["present_domains"]=df["present_domains"].apply(lambda row: len(row.split("\t")) if row!="" else 0).astype("uint8")
gc.collect()

0

##Drop columns i don't use

In [ ]:
df.drop("tweet_id",axis=1,inplace=True)
df.drop("engaging_user_account_creation",axis=1,inplace=True)
df.drop("engaged_with_user_account_creation",axis=1,inplace=True)
df.drop("tweet_timestamp",axis=1,inplace=True)
gc.collect()

50

##Check types

In [ ]:
df.dtypes

text_tokens                          uint16
hashtags                              uint8
present_media                         uint8
present_links                         uint8
present_domains                       uint8
tweet_type                           object
engaged_with_user_id                 object
engaged_with_user_follower_count      int64
engaged_with_user_following_count     int64
engaged_with_user_is_verified          bool
engaging_user_id                     object
engaging_user_follower_count          int64
engaging_user_following_count         int64
engaging_user_is_verified              bool
engagee_follows_engager                bool
like                                  uint8
retweet                               uint8
retweetCom                            uint8
reply                                 uint8
interaction                           uint8
photos                                uint8
gifs                                  uint8
videos                          

##Display head

In [ ]:
df.head()

,text_tokens,hashtags,present_media,present_links,present_domains,tweet_type,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos
0,50,0,0,0,0,Retweet,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0
1,78,0,0,1,1,TopLevel,C0EBA033A5C0D2AD91B4500958E2C633,4418640,228,True,865C52D28FA84E5E233B530C6FDAECC2,393,1190,False,False,0,0,0,0,0,0,0,0
2,39,0,0,1,1,TopLevel,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0
3,40,0,0,0,0,Retweet,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0
4,66,0,1,0,0,TopLevel,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0


In [ ]:
df.describe()

,text_tokens,hashtags,present_media,present_links,present_domains,engaged_with_user_follower_count,engaged_with_user_following_count,engaging_user_follower_count,engaging_user_following_count,like,retweet,retweetCom,reply,interaction,photos,gifs,videos
count,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06,3.033902e+06
mean,4.577666e+01,3.958948e-01,5.541504e-01,1.160591e-01,1.160591e-01,7.528097e+05,3.622267e+03,8.012008e+02,7.056084e+02,3.980313e-01,8.751766e-02,7.051975e-03,2.962027e-02,5.222212e-01,4.716405e-01,1.259269e-02,6.991722e-02
std,2.930063e+01,1.115992e+00,8.919881e-01,3.428927e-01,3.428927e-01,4.450692e+06,3.318459e+04,9.863545e+03,1.672492e+03,4.894920e-01,2.825922e-01,8.367943e-02,1.695374e-01,5.457068e-01,8.931532e-01,1.115202e-01,2.550682e-01
min,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.180000e+02,2.030000e+02,7.000000e+01,1.530000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.900000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.607000e+03,5.660000e+02,2.100000e+02,3.340000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.900000e+01,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,8.824000e+04,1.475000e+03,5.760000e+02,7.330000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,4.240000e+02,4.300000e+01,4.000000e+00,1.000000e+01,1.000000e+01,1.292405e+08,4.256371e+06,5.358904e+06,9.329220e+05,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,4.000000e+00,3.000000e+00,4.000000e+00


##Split dataset

In [ ]:
 msk = np.random.rand(len(df)) < 0.8
 
 train=df[msk]

 test=df[~msk]

In [ ]:
train.to_csv("train.csv", header=train.columns)
test.to_csv("test.csv", header=test.columns)

In [ ]:
#import pickle
#with open('ColNames.pkl', 'wb') as f:
#    pickle.dump(train.columns, f)


In [ ]:
!bzip2 -c --best train.csv > train.bz2
!bzip2 -c --best test.csv > test.bz2

In [ ]:
!cp train.bz2 "drive/MyDrive/twitter/one chunk/train.bz2"
!cp test.bz2 "drive/MyDrive/twitter/one chunk/test.bz2"
#!cp ColNames.pkl "drive/MyDrive/twitter/one chunk/ColNames.pkl"

#Prepare for Cugraph

In [ ]:
trainCu=pd.DataFrame(train.loc[df["interaction"]>0])


In [ ]:
for col in trainCu.columns:
    if col not in ["engaged_with_user_id","engaging_user_id","interaction"]:
        trainCu.drop(col,axis=1,inplace=True)

In [ ]:
trainCu["interactions"]=trainCu.groupby(["engaged_with_user_id","engaging_user_id"])['interaction'].transform('sum')

In [ ]:
trainCu= trainCu.drop_duplicates(subset= ["engaged_with_user_id","engaging_user_id"]).drop("interaction",axis=1)

In [ ]:
trainCu[:5]

,engaged_with_user_id,engaging_user_id,interactions
4,B7E441F2D886830FB3F0CE29CB35CB6B,04F45C4A769FE2D7C792A96C92342DC2,1
7,223797E039AFAC669C3A24A43B59CD5E,0F7B9515737C440274E43E09E01FF8CF,1
9,02EA65880A80121A394AB9CD652D7AD6,D3DE61AA5DFB88CEFBC2C9E675F40723,2
10,0A114052FE4DD3F5F7D2589673EC5ED8,0D07725F473577EE50C4EE831FC66D99,1
11,503B6E875783CABBC0F4CB7FEEA54622,C63E79394F6A9CCB4D2F85A200083F6E,1


In [ ]:
trainCu.columns

Index(['engaged_with_user_id', 'engaging_user_id', 'interactions'], dtype='object')

In [ ]:
trainCu.to_csv("trainCu.csv", header=trainCu.columns)

In [ ]:
!bzip2 -c --best trainCu.csv > trainCu.bz2

In [ ]:
!cp trainCu.bz2 "drive/MyDrive/twitter/onechunk/trainCu.bz2"

#After CuGraph

In [ ]:
!cp  drive/MyDrive/twitter/onechunk/* ../content/

In [ ]:
import os
files=os.listdir()
files=[f for f in files if f.endswith(".csv")]
files

['over.csv',
 'atlas-ing-to-ed.csv',
 'hits.csv',
 'pagerank.csv',
 'katz_centrality.csv',
 'jaccard-ing-to-ed.csv',
 'atlas-ed-to-ing.csv',
 'jaccard-ed-to-ing.csv']

In [ ]:
for fileT in files:
    with open(fileT) as f:
        print(f"{fileT}:{f.readline()}",end="")

over.csv:,overlap_coeff,source,destination
atlas-ing-to-ed.csv:,x,y,vertex
hits.csv:,hubs,authorities,vertex
pagerank.csv:,pagerank,vertex
katz_centrality.csv:,katz_centrality,vertex
jaccard-ing-to-ed.csv:,jaccard_coeff,source,destination
atlas-ed-to-ing.csv:,x,y,vertex
jaccard-ed-to-ing.csv:,jaccard_coeff,source,destination


In [ ]:
!bunzip2 train.bz2
!mv train train.csv

In [ ]:
import pandas as pd
train=pd.read_csv("train.csv", index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
df=train
train[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel
0,50,0,0,0,0,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0,0,1,0
2,39,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0,0,0,1
3,40,0,0,0,0,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0,0,1,0
4,66,0,1,0,0,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0,0,0,1
5,3,0,0,0,0,313ECD3A1E5BB07406E4249475C2D6D6,19DF812A5A26D6224CB08F6DEC005E93,673,561,False,7785D49310219E087001E5E62FA67893,2322,996,False,True,0,0,0,0,0,0,0,0,1,0,0


#ADD pagerank

In [ ]:
pagerank_ed_to_ing=pd.read_csv("pagerank.csv",index_col=0)


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
pagerank_ed_to_ing.columns=["pagerank_ed_to_ing","engaged_with_user_id"]

In [ ]:
pagerank_ed_to_ing[:5]

,pagerank_ed_to_ing,engaged_with_user_id
1131207,0.000280,1DE5D18797E212A3850C5FB5C35E12E2
1519173,0.000207,B4246E7A34B52630ED396609E307F784
140261,0.000189,9C3AAFC3C2C2F129065F4BB5B00426B4
1511093,0.000184,F050BCE6C6A405C11C8D7897017CE1C1
1174528,0.000171,D956CB05C8566B63F75C86A595A84417


In [ ]:
df=df.merge(pagerank_ed_to_ing,on="engaged_with_user_id",how="left")

In [ ]:
df["pagerank_ed_to_ing_ed"]=df["pagerank_ed_to_ing"].apply(lambda x: x if x>0 else 0)
df.drop("pagerank_ed_to_ing",axis=1,inplace=True)

In [ ]:
pagerank_ed_to_ing.columns=["pagerank_ed_to_ing","engaging_user_id"]

In [ ]:
df=df.merge(pagerank_ed_to_ing,on="engaging_user_id",how="left")

In [ ]:
df["pagerank_ed_to_ing_ing"]=df["pagerank_ed_to_ing"].apply(lambda x: x if x>0 else 0)
df.drop("pagerank_ed_to_ing",axis=1,inplace=True)


In [ ]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing
0,50,0,0,0,0,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,2.967755e-07
1,39,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0,0,0,1,1.565519e-05,3.329616e-07
2,40,0,0,0,0,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0,0,1,0,1.509843e-06,0.000000e+00
3,66,0,1,0,0,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0,0,0,1,6.348989e-07,6.348989e-07
4,3,0,0,0,0,313ECD3A1E5BB07406E4249475C2D6D6,19DF812A5A26D6224CB08F6DEC005E93,673,561,False,7785D49310219E087001E5E62FA67893,2322,996,False,True,0,0,0,0,0,0,0,0,1,0,0,0.000000e+00,0.000000e+00


In [ ]:
del pagerank_ed_to_ing

#ADD HITS

In [ ]:
hits_ed_to_ing=pd.read_csv("hits.csv",index_col=0)


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
hits_ed_to_ing.columns=["hubs_ed","authorities_ed","engaged_with_user_id"]

In [ ]:
hits_ed_to_ing[:5]

,hubs_ed,authorities_ed,engaged_with_user_id
0,1.165709e-09,1.165716e-09,02FA3E768BA51EE08AA9D098ED079482
1,3.879835e-12,3.879881e-12,B47FCA079E5D01F901B819ECE263F1DD
2,0.000000e+00,0.000000e+00,E36A5DA720EF476EC61F79B753CCEDA5
3,7.597836e-14,7.598496e-14,15FCC988F9C819109E20D31BE923A578
4,2.612996e-11,2.613013e-11,98C1837F4BAA52B93B45E0AF8C375C2F


In [ ]:
df=df.merge(hits_ed_to_ing,on="engaged_with_user_id",how="left")

In [ ]:
df["hubs_ed"]=df["hubs_ed"].apply(lambda x: x if x>0 else 0)
df["authorities_ed"]=df["authorities_ed"].apply(lambda x: x if x>0 else 0)

In [ ]:
hits_ed_to_ing.columns=["hubs_ing","authorities_ing","engaging_user_id"]

In [ ]:
df=df.merge(hits_ed_to_ing,on="engaging_user_id",how="left")

In [ ]:
df["hubs_ing"]=df["hubs_ing"].apply(lambda x: x if x>0 else 0)
df["authorities_ing"]=df["authorities_ing"].apply(lambda x: x if x>0 else 0)

In [ ]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing
0,50,0,0,0,0,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,2.967755e-07,0.0,0.0,1.098416e-12,1.098423e-12
1,39,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0,0,0,1,1.565519e-05,3.329616e-07,0.0,0.0,0.000000e+00,0.000000e+00
2,40,0,0,0,0,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0,0,1,0,1.509843e-06,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00
3,66,0,1,0,0,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0,0,0,1,6.348989e-07,6.348989e-07,0.0,0.0,0.000000e+00,0.000000e+00
4,3,0,0,0,0,313ECD3A1E5BB07406E4249475C2D6D6,19DF812A5A26D6224CB08F6DEC005E93,673,561,False,7785D49310219E087001E5E62FA67893,2322,996,False,True,0,0,0,0,0,0,0,0,1,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00


In [ ]:
del hits_ed_to_ing

#ADD Katz

In [ ]:
katz_ed_to_ing=pd.read_csv("katz_centrality.csv",index_col=0)


/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
katz_ed_to_ing.columns=["katz_ed","engaged_with_user_id"]

In [ ]:
katz_ed_to_ing[:5]

,katz_ed,engaged_with_user_id
0,0.001637,D929B501E45AABD4E2AF47745D272157
1,0.004231,6FADF44AFDC89F764E2AF553E814A3AD
2,0.001586,CE02EC14E2D64DF46EF87B2E0244B415
3,0.003249,FB35EA040622F3C04AE9E1EA52147FE6
4,0.001755,C6149C2908123D272325E5C0D67F5BF0


In [ ]:
df=df.merge(katz_ed_to_ing,on="engaged_with_user_id",how="left")

In [ ]:
df["katz_ed"]=df["katz_ed"].apply(lambda x: x if x>0 else 0)

In [ ]:
katz_ed_to_ing.columns=["katz_ing","engaging_user_id"]

In [ ]:
df=df.merge(katz_ed_to_ing,on="engaging_user_id",how="left")

In [ ]:
df["katz_ing"]=df["katz_ing"].apply(lambda x: x if x>0 else 0)

In [ ]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing
0,50,0,0,0,0,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,2.967755e-07,0.0,0.0,1.098416e-12,1.098423e-12,0.000000,0.000178
1,39,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0,0,0,1,1.565519e-05,3.329616e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.009320,0.000185
2,40,0,0,0,0,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0,0,1,0,1.509843e-06,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000703,0.000000
3,66,0,1,0,0,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0,0,0,1,6.348989e-07,6.348989e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.000176,0.000176
4,3,0,0,0,0,313ECD3A1E5BB07406E4249475C2D6D6,19DF812A5A26D6224CB08F6DEC005E93,673,561,False,7785D49310219E087001E5E62FA67893,2322,996,False,True,0,0,0,0,0,0,0,0,1,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000


In [ ]:
del katz_ed_to_ing

#ADD Jaccard

##ADD Jaccard to the dataframe

In [ ]:
jaccard_ed_to_ing=pd.read_csv("jaccard-ed-to-ing.csv",index_col=0)


In [ ]:
jaccard_ed_to_ing.columns=["jaccard_ed_to_ing_ed","engaged_with_user_id","engaging_user_id"]

In [ ]:
jaccard_ed_to_ing[:5]

,jaccard_ed_to_ing_ed,engaged_with_user_id,engaging_user_id
713697,0.666667,5CB68E06FAF45602C095AE5964D2B4C2,1E310376604254152F0DDC60D0093347
2284025,0.666667,1E310376604254152F0DDC60D0093347,5CB68E06FAF45602C095AE5964D2B4C2
554994,0.600000,DAA3B34033B99FC4BCCF456EEB073D40,FAE7207C477F9B680174EDC501A76C44
1829080,0.600000,FAE7207C477F9B680174EDC501A76C44,DAA3B34033B99FC4BCCF456EEB073D40
458055,0.500000,67ECB116DEC459EFD4C8F1199370BE17,7D8560C4125CC618C34515689DD8D9E4


In [ ]:
df=df.merge(jaccard_ed_to_ing,on=["engaged_with_user_id","engaging_user_id"],how="left")

In [ ]:
df["jaccard_ed_to_ing_ed"]=df["jaccard_ed_to_ing_ed"].apply(lambda x: x if x>0 else 0)


In [ ]:
jaccard_ed_to_ing.columns=["jaccard_ed_to_ing_ing","engaging_user_id","engaged_with_user_id"]

In [ ]:
df=df.merge(jaccard_ed_to_ing,on=["engaging_user_id","engaged_with_user_id"],how="left")

In [ ]:
df["jaccard_ed_to_ing_ing"]=df["jaccard_ed_to_ing_ing"].apply(lambda x: x if x>0 else 0)


In [ ]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing,jaccard_ed_to_ing_ed,jaccard_ed_to_ing_ing
0,50,0,0,0,0,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,2.967755e-07,0.0,0.0,1.098416e-12,1.098423e-12,0.000000,0.000178,0.0,0.0
1,39,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0,0,0,1,1.565519e-05,3.329616e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.009320,0.000185,0.0,0.0
2,40,0,0,0,0,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0,0,1,0,1.509843e-06,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000703,0.000000,0.0,0.0
3,66,0,1,0,0,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0,0,0,1,6.348989e-07,6.348989e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.000176,0.000176,0.0,0.0
4,3,0,0,0,0,313ECD3A1E5BB07406E4249475C2D6D6,19DF812A5A26D6224CB08F6DEC005E93,673,561,False,7785D49310219E087001E5E62FA67893,2322,996,False,True,0,0,0,0,0,0,0,0,1,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0


In [ ]:
del jaccard_ed_to_ing

##ADD Jaccard to the dataframe opposite

In [ ]:
jaccard_ing_to_ed=pd.read_csv("jaccard-ing-to-ed.csv",index_col=0)


In [ ]:
jaccard_ing_to_ed.columns=["jaccard_ing_to_ed_ed","engaged_with_user_id","engaging_user_id"]

In [ ]:
jaccard_ing_to_ed[:5]

,jaccard_ing_to_ed_ed,engaged_with_user_id,engaging_user_id
263536,0.666667,5BCD16926EDF8B3A66E7D658AC70513D,06C5FD59BD965EE20CBA4F7D4D7F7859
280054,0.666667,69B871CC7FEE3B5381941B7F71006C82,EC478D67017FCEF8CA658CA3CECE2C47
306050,0.666667,723DB181D6E6B38379B8B53E2C098ECC,62AF7A8F6A3DA4B3FDE7D0FBDE1477FA
917929,0.666667,8C69FBD2209B0F1DE08ABB50BBADEF6F,B3733C09AD13E0B01D0134FB3CCAFE44
1321244,0.666667,2641FBB1227B81448E9432B2823BFB7E,CD33A2E2CC5284196899F5CC87E74D11


In [ ]:
df=df.merge(jaccard_ing_to_ed,on=["engaged_with_user_id","engaging_user_id"],how="left")

In [ ]:
df["jaccard_ing_to_ed_ed"]=df["jaccard_ing_to_ed_ed"].apply(lambda x: x if x>0 else 0)


In [ ]:
jaccard_ing_to_ed.columns=["jaccard_ing_to_ed_ing","engaging_user_id","engaged_with_user_id"]

In [ ]:
df=df.merge(jaccard_ing_to_ed,on=["engaging_user_id","engaged_with_user_id"],how="left")

In [ ]:
df["jaccard_ing_to_ed_ing"]=df["jaccard_ing_to_ed_ing"].apply(lambda x: x if x>0 else 0)


In [ ]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing,jaccard_ed_to_ing_ed,jaccard_ed_to_ing_ing,jaccard_ing_to_ed_ed,jaccard_ing_to_ed_ing
0,50,0,0,0,0,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,2.967755e-07,0.0,0.0,1.098416e-12,1.098423e-12,0.000000,0.000178,0.0,0.0,0.0,0.0
1,39,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0,0,0,1,1.565519e-05,3.329616e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.009320,0.000185,0.0,0.0,0.0,0.0
2,40,0,0,0,0,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0,0,1,0,1.509843e-06,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000703,0.000000,0.0,0.0,0.0,0.0
3,66,0,1,0,0,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0,0,0,1,6.348989e-07,6.348989e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.000176,0.000176,0.0,0.0,0.0,0.0
4,3,0,0,0,0,313ECD3A1E5BB07406E4249475C2D6D6,19DF812A5A26D6224CB08F6DEC005E93,673,561,False,7785D49310219E087001E5E62FA67893,2322,996,False,True,0,0,0,0,0,0,0,0,1,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0,0.0


In [ ]:
del jaccard_ing_to_ed

#ADD Overlap

In [ ]:
overlap_ed_to_ing=pd.read_csv("over.csv",index_col=0)


In [ ]:
overlap_ed_to_ing.columns=["overlap_ed","engaged_with_user_id","engaging_user_id"]

In [ ]:
overlap_ed_to_ing[:5]

,overlap_ed,engaged_with_user_id,engaging_user_id
9431,0.666667,56238EC8CB5C51DA10E228C7D4D395A4,2BE23D1A601A50E2D007C6F0090CF091
15942,0.666667,DBA3D392CFA98ED80D4D35CFFAFAB0CB,66A770D1BE1D360AF2A33F024284295E
120536,0.666667,CA33EBA30174D470C5A2189D96D668F3,7B6A39F2FE6D377A75F857E1837DD8B8
148176,0.666667,2BE23D1A601A50E2D007C6F0090CF091,56238EC8CB5C51DA10E228C7D4D395A4
178048,0.666667,93207183C9BFD46714F749AF05C0246A,B474FE65F1C6025D8DED4D3C5FADC11E


In [ ]:
df=df.merge(overlap_ed_to_ing,on=["engaged_with_user_id","engaging_user_id"],how="left")

In [ ]:
df["overlap_ed"]=df["overlap_ed"].apply(lambda x: x if x>0 else 0)


In [ ]:
overlap_ed_to_ing.columns=["overlap_ing","engaging_user_id","engaged_with_user_id"]

In [ ]:
df=df.merge(overlap_ed_to_ing,on=["engaging_user_id","engaged_with_user_id"],how="left")

In [ ]:
df["overlap_ing"]=df["overlap_ing"].apply(lambda x: x if x>0 else 0)


In [ ]:
def f(row):
    e=row["overlap_ed"]
    i=row["overlap_ing"]
    if e >0:
        if i>0:
            return (e+i)/2
        else:
            return e
    else:
        if i>0:
            return i
        else:
            return 0
df["overlap"]=df.apply(f, axis=1)

df.drop("overlap_ed",axis=1,inplace=True)
df.drop("overlap_ing",axis=1,inplace=True)

In [ ]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing,jaccard_ed_to_ing_ed,jaccard_ed_to_ing_ing,jaccard_ing_to_ed_ed,jaccard_ing_to_ed_ing,overlap
0,50,0,0,0,0,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,2.967755e-07,0.0,0.0,1.098416e-12,1.098423e-12,0.000000,0.000178,0.0,0.0,0.0,0.0,0.0
1,39,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0,0,0,1,1.565519e-05,3.329616e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.009320,0.000185,0.0,0.0,0.0,0.0,0.0
2,40,0,0,0,0,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0,0,1,0,1.509843e-06,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000703,0.000000,0.0,0.0,0.0,0.0,0.0
3,66,0,1,0,0,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0,0,0,1,6.348989e-07,6.348989e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.000176,0.000176,0.0,0.0,0.0,0.0,0.0
4,3,0,0,0,0,313ECD3A1E5BB07406E4249475C2D6D6,19DF812A5A26D6224CB08F6DEC005E93,673,561,False,7785D49310219E087001E5E62FA67893,2322,996,False,True,0,0,0,0,0,0,0,0,1,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [ ]:
del overlap_ed_to_ing

#Use datafrme

##Fix dataset

In [ ]:
istrain=True

##Check if talk that language

In [ ]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing,jaccard_ed_to_ing_ed,jaccard_ed_to_ing_ing,jaccard_ing_to_ed_ed,jaccard_ing_to_ed_ing,overlap
0,50,0,0,0,0,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,2.967755e-07,0.0,0.0,1.098416e-12,1.098423e-12,0.000000,0.000178,0.0,0.0,0.0,0.0,0.0
1,39,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0,0,0,1,1.565519e-05,3.329616e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.009320,0.000185,0.0,0.0,0.0,0.0,0.0
2,40,0,0,0,0,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0,0,1,0,1.509843e-06,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000703,0.000000,0.0,0.0,0.0,0.0,0.0
3,66,0,1,0,0,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0,0,0,1,6.348989e-07,6.348989e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.000176,0.000176,0.0,0.0,0.0,0.0,0.0
4,3,0,0,0,0,313ECD3A1E5BB07406E4249475C2D6D6,19DF812A5A26D6224CB08F6DEC005E93,673,561,False,7785D49310219E087001E5E62FA67893,2322,996,False,True,0,0,0,0,0,0,0,0,1,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [ ]:
import gc
gc.collect()

377

In [ ]:

t=df[["engaging_user_id","language","interaction"]].copy()
lang=t
langDir=lang[lang["interaction"]>0]
langDir.drop("interaction",inplace=True,axis=1)
langDir.columns=["id","language"]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
langDir[:5]

,id,language
1,FE83E568BD4B2ACDBAF876CFBB6C7296,B0FA488F2911701DD8EC5B1EA5E322D8
3,04F45C4A769FE2D7C792A96C92342DC2,E7F038DE3EAD397AEC9193686C911677
6,0F7B9515737C440274E43E09E01FF8CF,488B32D24BD4BB44172EB981C1BCA6FA
8,C63E79394F6A9CCB4D2F85A200083F6E,488B32D24BD4BB44172EB981C1BCA6FA
9,7EE0A31E5381C7CF4A572F29F6FBE625,488B32D24BD4BB44172EB981C1BCA6FA


In [ ]:
t=df[["engaged_with_user_id","language","interaction"]].copy()
lang=t
langInv=lang[lang["interaction"]>0]
langInv.drop("interaction",inplace=True,axis=1)
langInv.columns=["id","language"]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
langInv[:5]

,id,language
1,03EA05E7440140ABF047BC216CE34917,B0FA488F2911701DD8EC5B1EA5E322D8
3,B7E441F2D886830FB3F0CE29CB35CB6B,E7F038DE3EAD397AEC9193686C911677
6,223797E039AFAC669C3A24A43B59CD5E,488B32D24BD4BB44172EB981C1BCA6FA
8,503B6E875783CABBC0F4CB7FEEA54622,488B32D24BD4BB44172EB981C1BCA6FA
9,D77A8DF6804617BFD165C4A11304A27A,488B32D24BD4BB44172EB981C1BCA6FA


In [ ]:
lang=pd.concat([langDir,langInv])

In [ ]:
lang[:5]

,id,language
1,FE83E568BD4B2ACDBAF876CFBB6C7296,B0FA488F2911701DD8EC5B1EA5E322D8
3,04F45C4A769FE2D7C792A96C92342DC2,E7F038DE3EAD397AEC9193686C911677
6,0F7B9515737C440274E43E09E01FF8CF,488B32D24BD4BB44172EB981C1BCA6FA
8,C63E79394F6A9CCB4D2F85A200083F6E,488B32D24BD4BB44172EB981C1BCA6FA
9,7EE0A31E5381C7CF4A572F29F6FBE625,488B32D24BD4BB44172EB981C1BCA6FA


In [ ]:
del langDir
del langInv

In [ ]:
lang.insert(2,"understands",1)
lang.columns=["engaging_user_id","language","understands"]
lang=lang.groupby(["engaging_user_id","language"]).sum()

In [ ]:
lang[10:15]

,,understands
engaging_user_id,language,
00006D6ABF1482658965CCB1B1025001,B0FA488F2911701DD8EC5B1EA5E322D8,1
00006D6D3F592C42682AFF06D22FB190,488B32D24BD4BB44172EB981C1BCA6FA,2
000079314D412C5A0B768A7B8C5817FE,1F73BB863A39DB62B4A55B7E558DB1E8,1
0000B15CBCE400468637050BAC69BE10,B0FA488F2911701DD8EC5B1EA5E322D8,1
0000B5E1234F983C48FC4AB477FB9A7D,488B32D24BD4BB44172EB981C1BCA6FA,1


In [ ]:
lang["understands"]=lang["understands"].apply(lambda x: (x>1) if istrain else x>0)

In [ ]:
lang[10:15]

,,understands
engaging_user_id,language,
00006D6ABF1482658965CCB1B1025001,B0FA488F2911701DD8EC5B1EA5E322D8,False
00006D6D3F592C42682AFF06D22FB190,488B32D24BD4BB44172EB981C1BCA6FA,True
000079314D412C5A0B768A7B8C5817FE,1F73BB863A39DB62B4A55B7E558DB1E8,False
0000B15CBCE400468637050BAC69BE10,B0FA488F2911701DD8EC5B1EA5E322D8,False
0000B5E1234F983C48FC4AB477FB9A7D,488B32D24BD4BB44172EB981C1BCA6FA,False


In [ ]:
temp=lang[lang["understands"]].copy()

In [ ]:
temp[:5]

,,understands
engaging_user_id,language,
00006D6D3F592C42682AFF06D22FB190,488B32D24BD4BB44172EB981C1BCA6FA,True
000174419CBFFB135857CF1B52A5050F,9A78FC330083E72BE0DD1EA92656F3B5,True
0001B88EA47CAD14F55B97BDBE4F0D44,E7F038DE3EAD397AEC9193686C911677,True
0001F7A2768F7BEEEDB0924F5E122D39,C1E99BF67DDA2227007DE8038FE32470,True
000213BA3D2389BEA0BECF1B72994518,E7F038DE3EAD397AEC9193686C911677,True


In [ ]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing,jaccard_ed_to_ing_ed,jaccard_ed_to_ing_ing,jaccard_ing_to_ed_ed,jaccard_ing_to_ed_ing,overlap
0,50,0,0,0,0,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,2.967755e-07,0.0,0.0,1.098416e-12,1.098423e-12,0.000000,0.000178,0.0,0.0,0.0,0.0,0.0
1,39,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0,0,0,1,1.565519e-05,3.329616e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.009320,0.000185,0.0,0.0,0.0,0.0,0.0
2,40,0,0,0,0,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0,0,1,0,1.509843e-06,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000703,0.000000,0.0,0.0,0.0,0.0,0.0
3,66,0,1,0,0,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0,0,0,1,6.348989e-07,6.348989e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.000176,0.000176,0.0,0.0,0.0,0.0,0.0
4,3,0,0,0,0,313ECD3A1E5BB07406E4249475C2D6D6,19DF812A5A26D6224CB08F6DEC005E93,673,561,False,7785D49310219E087001E5E62FA67893,2322,996,False,True,0,0,0,0,0,0,0,0,1,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [ ]:

df=df.merge(temp,on=["engaging_user_id","language"],how="left")


In [ ]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing,jaccard_ed_to_ing_ed,jaccard_ed_to_ing_ing,jaccard_ing_to_ed_ed,jaccard_ing_to_ed_ing,overlap,understands
0,50,0,0,0,0,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,2.967755e-07,0.0,0.0,1.098416e-12,1.098423e-12,0.000000,0.000178,0.0,0.0,0.0,0.0,0.0,NaN
1,39,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0,0,0,1,1.565519e-05,3.329616e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.009320,0.000185,0.0,0.0,0.0,0.0,0.0,NaN
2,40,0,0,0,0,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0,0,1,0,1.509843e-06,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000703,0.000000,0.0,0.0,0.0,0.0,0.0,NaN
3,66,0,1,0,0,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0,0,0,1,6.348989e-07,6.348989e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.000176,0.000176,0.0,0.0,0.0,0.0,0.0,NaN
4,3,0,0,0,0,313ECD3A1E5BB07406E4249475C2D6D6,19DF812A5A26D6224CB08F6DEC005E93,673,561,False,7785D49310219E087001E5E62FA67893,2322,996,False,True,0,0,0,0,0,0,0,0,1,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,NaN


In [ ]:
df["understands"]=df["understands"].fillna(False)

In [ ]:
df[:5]

,text_tokens,hashtags,present_media,present_links,present_domains,language,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engagee_follows_engager,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing,jaccard_ed_to_ing_ed,jaccard_ed_to_ing_ing,jaccard_ing_to_ed_ed,jaccard_ing_to_ed_ing,overlap,understands
0,50,0,0,0,0,488B32D24BD4BB44172EB981C1BCA6FA,BC9980C5385A68894C2862F41884FB27,2473,662,False,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,False,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,2.967755e-07,0.0,0.0,1.098416e-12,1.098423e-12,0.000000,0.000178,0.0,0.0,0.0,0.0,0.0,False
1,39,0,0,1,1,B0FA488F2911701DD8EC5B1EA5E322D8,03EA05E7440140ABF047BC216CE34917,219715,3685,True,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,False,1,0,0,0,1,0,0,0,0,0,1,1.565519e-05,3.329616e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.009320,0.000185,0.0,0.0,0.0,0.0,0.0,False
2,40,0,0,0,0,B0FA488F2911701DD8EC5B1EA5E322D8,2700B9E175757F126C082D6A5742787D,2388283,13511,True,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,False,0,0,0,0,0,0,0,0,0,1,0,1.509843e-06,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000703,0.000000,0.0,0.0,0.0,0.0,0.0,False
3,66,0,1,0,0,E7F038DE3EAD397AEC9193686C911677,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,True,1,0,0,0,1,1,0,0,0,0,1,6.348989e-07,6.348989e-07,0.0,0.0,0.000000e+00,0.000000e+00,0.000176,0.000176,0.0,0.0,0.0,0.0,0.0,False
4,3,0,0,0,0,313ECD3A1E5BB07406E4249475C2D6D6,19DF812A5A26D6224CB08F6DEC005E93,673,561,False,7785D49310219E087001E5E62FA67893,2322,996,False,True,0,0,0,0,0,0,0,0,1,0,0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,False


In [ ]:
del lang
del temp

##LastPreprocess

In [ ]:
df.drop("language",axis=1,inplace=True)

In [ ]:
df.drop("engaged_with_user_id",axis=1,inplace=True)
df.drop("engaging_user_id",axis=1,inplace=True)
df.drop("interaction",axis=1,inplace=True)

In [ ]:
df.describe()

,text_tokens,hashtags,present_media,present_links,present_domains,engaged_with_user_follower_count,engaged_with_user_following_count,engaging_user_follower_count,engaging_user_following_count,like,retweet,retweetCom,reply,interaction,photos,gifs,videos,tweet_type_Quote,tweet_type_Retweet,tweet_type_TopLevel,pagerank_ed_to_ing_ed,pagerank_ed_to_ing_ing,hubs_ed,authorities_ed,hubs_ing,authorities_ing,katz_ed,katz_ing,jaccard_ed_to_ing_ed,jaccard_ed_to_ing_ing,jaccard_ing_to_ed_ed,jaccard_ing_to_ed_ing,overlap
count,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06,2.426667e+06
mean,4.578164e+01,3.960358e-01,5.542396e-01,1.160345e-01,1.160345e-01,7.527102e+05,3.621400e+03,8.020131e+02,7.059730e+02,3.982372e-01,8.749655e-02,7.050823e-03,2.963159e-02,5.224161e-01,4.716102e-01,1.265769e-02,6.997169e-02,8.147966e-02,3.279544e-01,5.905660e-01,4.836986e-06,4.713653e-07,1.056866e-06,1.056768e-06,9.400719e-07,9.400721e-07,3.222801e-03,2.339061e-04,9.060235e-05,9.060235e-05,8.847196e-05,8.847196e-05,2.553087e-04
std,2.930252e+01,1.116160e+00,8.919822e-01,3.427870e-01,3.427870e-01,4.456880e+06,3.292555e+04,9.616783e+03,1.673205e+03,4.895349e-01,2.825614e-01,8.367265e-02,1.695688e-01,5.458088e-01,8.931478e-01,1.118069e-01,2.551720e-01,2.735704e-01,4.694682e-01,4.917295e-01,1.851352e-05,7.830783e-07,4.211121e-05,4.210721e-05,3.054176e-05,3.054177e-05,1.260607e-02,4.977579e-04,4.520509e-03,4.520509e-03,4.678667e-03,4.678667e-03,1.041358e-02
min,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.190000e+02,2.030000e+02,7.000000e+01,1.530000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.900000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.605000e+03,5.660000e+02,2.100000e+02,3.340000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,6.348989e-07,3.594222e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.760200e-04,1.758380e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,5.900000e+01,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,8.810000e+04,1.475000e+03,5.760000e+02,7.330000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.633369e-06,6.348989e-07,3.631541e-17,3.631395e-17,2.259484e-21,2.259336e-21,8.819300e-04,1.932980e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,4.240000e+02,4.300000e+01,4.000000e+00,1.000000e+01,1.000000e+01,1.292405e+08,4.256371e+06,5.358904e+06,9.329220e+05,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,4.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.797440e-04,1.893350e-04,1.738119e-03,1.737947e-03,1.021675e-03,1.021675e-03,1.756172e-01,1.164934e-01,6.666667e-01,6.666667e-01,6.666667e-01,6.666667e-01,6.666667e-01


##For Like


In [ ]:
df.drop("retweet",axis=1,inplace=True)
df.drop("retweetCom",axis=1,inplace=True)
df.drop("reply",axis=1,inplace=True)
target="like"

##For ret


In [ ]:
df.drop("like",axis=1,inplace=True)
df.drop("retweetCom",axis=1,inplace=True)
df.drop("reply",axis=1,inplace=True)
target="retweet"

##For quote

In [ ]:
df.drop("retweet",axis=1,inplace=True)
df.drop("like",axis=1,inplace=True)
df.drop("reply",axis=1,inplace=True)
target="retweetCom"

##For Reply

In [ ]:
df.drop("retweet",axis=1,inplace=True)
df.drop("retweetCom",axis=1,inplace=True)
df.drop("like",axis=1,inplace=True)
target="reply"

#Model

In [ ]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
targetCol= train[target]
df.drop(target,axis=1,inplace=True)

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.metrics import mean_squared_error
train=df.values
target=targetCol.values
del df
del targetCol
import gc
gc.collect()

150

In [ ]:
xgbc = xgb.XGBClassifier()

In [ ]:
xgbc.fit(train, target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
trainPred=xgbc.predict(train)


In [ ]:
cm = confusion_matrix(target,trainPred)
cm

array([[1217669,  242609],
       [    927,  965462]])

In [ ]:
scores = cross_val_score(xgbc, train, target, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())

In [ ]:
print(xgbc.feature_importances_)

In [ ]:
xgb.plot_importance(xgbc)

In [ ]:
xgbc.save_model('model_Like.json')

In [ ]:
!cp model_Like.json drive/MyDrive/model_like.json

In [ ]:
train.columns

#Close On complete

In [ ]:
!kill -9 -1